# EXERCISE SOLUTION

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

In [3]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [4]:
# Constants

MODEL = "llama3.2"

In [5]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# Let's try one out

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
AI Curriculum
Proficient AI Engineer
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
I will happily drone on for hours about LLMs to anyone in my vicinity. My friends got fed up with my impromptu lectures, and convinced me to make some Udemy courses. To my total joy (and shock) they’ve become best-selling, top-rated courses, with 400,000 enrolled across 190 

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from Ollama expects the same message format as OpenAI:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [9]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - now with Ollama instead of OpenAI

In [10]:
# And now: call the Ollama function instead of OpenAI

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [11]:
summarize("https://edwarddonner.com")

'**Summary of the Website**\n==========================\n\n### Introduction\n\nThe website is run by Ed, a co-founder and CTO of Nebula.io, who applies AI to help people discover their potential. He also created Udemy courses on LLMs that have become best-sellers.\n\n### News and Announcements\n\n* **January 4, 2026**: Release of an "AI Builder with n8n" tutorial.\n* **November 11, 2025**: Announcement of a live event featuring AI.\n* **September 15, 2025**: Promotion of the AI Engineering MLOps Track.\n* **May 28, 2025**: Release of the AI courses.\n\n### About Ed\n\nEd is an enthusiast of LLMs and enjoys amateur electronic music production. He\'s also grateful for the success of his Udemy courses, which have been taken by over 400,000 people across 190 countries.\n\n### Contact Information\n\n* Email: [ed@edwarddonner.com](mailto:ed@edwarddonner.com)\n* Website: www.edwarddonner.com\n* Social Media:\n\t+ LinkedIn: Follow him on LinkedIn.\n\t+ Twitter: Follow him on Twitter.\n\t+ Face

In [12]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [18]:
display_summary("https://edwarddonner.com")

**Website Summary**
======================

### About the Creator

* Ed is the co-founder and CTO of Nebula.io, applying AI to help people discover their potential.
* He's also a founder and former CEO of untapt, an AI startup acquired in 2021.

### Recent News and Announcements

* **January 4, 2026**: "AI Builder with n8n – Create Agents and Voice Agents"
* **November 11, 2025**: "The Unique Energy of an AI Live Event"
* **September 15, 2025**: "AI Engineering MLOps Track – Deploy AI to Production"
* **May 28, 2025**: A blog post about the best order for taking Ed's AI courses.

### Other Notable Content

* A Udemy course curriculum with over 400,000 enrollments across 190 countries.
* Links to Ed's social media profiles (LinkedIn, Twitter, Facebook).

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [20]:
display_summary("https://cnn.com")

Here is a summary of the top news stories from around the world, as reported by CNN:

**Politics**

* The US midterm elections are still months away, but President Joe Biden is already facing pressure to cancel them or postpone them. However, sources say it's unlikely that he can do either.
* Trump's vetting team asked whether Shapiro was an "agent of the Israeli government" during his Senate confirmation process.
* Ukrainian presidential candidate Yulia Tymoshenko has been arrested and charged with tax evasion.

**World**

* Eight skiers have died in avalanches in Austria, with many more injured.
* A prehistoric wolf's gut has been frozen in time, revealing an ice age giant.
* China's birth rate has dropped to a record low as the country struggles to balance its economy and population growth.

**Business**

* The "Department of War" rebrand could cost up to $125 million, according to the Congressional Budget Office.
* Tesla CEO Elon Musk is suing a UK court over his xAI company.
* ChatGPT will start showing users ads based on their conversations.

**Health**

* A 4-year-old girl has died from flu complications, and her mother is speaking out about the importance of vaccinations.
* Chloe Kim says she's "good to go" for the Winter Olympics despite a labrum tear in her shoulder.

**Sports**

* The Los Angeles Fire Department crew tasked with protecting communities from wildfires is ready to deploy.

**Entertainment**

* Thousands are celebrating the life of legendary Grateful Dead guitarist Bob Weir.
* Finn Wolfhard was surprised by his Stranger Things co-stars on Saturday Night Live.

**Climate**

* Climate change is having a devastating impact on Ukraine, with extreme weather events and rising temperatures making it harder for people to survive.

**Business**

* Tesla CEO Elon Musk's xAI company is suing a UK court over its "deepfake" images.
* ChatGPT will start showing users ads based on their conversations.

I hope this summary provides a good overview of the top news stories from around the world.

In [15]:
display_summary("https://anthropic.com")

# Anthropic Website Summary

## Mission Statement

Anthropic is a public benefit corporation dedicated to securing the benefits and mitigating the risks of AI, aiming to build tools with human benefit at their foundation.

### Core Views on AI Safety

* Responsible AI development requires both bold steps forward and intentional pauses to consider effects.
* Anthropic focuses on building tools with human benefit at their foundation, like Claude.

## Research Areas

* Economic Futures
	+ Research on the economic impact of AI
* Initiatives
	+ Policy work and product design for responsible AI development
* Transparency
	+ Open data and research for public consumption
* Responsible Scaling Policy
	+ Guidelines for scaling AI projects with minimal risk

### Recent News and Announcements

* **Claude Opus 4.5**: The best model in the world for coding, agents, computer use, and enterprise workflows.
* **Introducing Haiku 4.5** (September 29, 2025)
* **Anthropic Economic Index** (September 15, 2025)
* **Claude Opus 4.1** (August 05, 2025)

## Products and Solutions

* Claude
	+ A coding tool for agents, computer use, and enterprise workflows
* Haiku
	+ A conversational AI platform
* Sonnet
	+ An explainable AI model

## Careers and Community Engagement

* **Open Roles**: See available positions to help build the future of safe AI.
* **Sales Support**: Connect with sales representatives for more information.

### Footer Information

* Anthropic PBC 2025

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

PR instructions courtesy of an AI friend: https://chatgpt.com/share/670145d5-e8a8-8012-8f93-39ee4e248b4c